## TASK 3 - Scene recognition with Bag of Visual Words

Group Number: #INSERT HERE


### 1 Data import and Preparation

Complete the class ***SceneDataset*** in ***dataset.py*** and load the training and test dataset. (images and class labels)

In [38]:
%load_ext autoreload
%autoreload 1
%aimport features, dataset, utils
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import numpy as np
import features
import dataset
import utils
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
if not os.path.exists(os.path.join(os.getcwd(), 'results')):
    os.makedirs(os.path.join(os.getcwd(), 'results'))

#student_code start

import dataset

datasetTrain = dataset.SceneDataset(path="data\\scene\\train")
datasetTest = dataset.SceneDataset(path="data\\scene\\test")

test_images = datasetTest.images
test_labels = datasetTest.labels

train_images = datasetTrain.images
train_labels = datasetTrain.labels

class_names = datasetTrain.class_names

# test_images = np.array(test_images)
# train_images = np.array(train_images)

#student_code end
print('Imported test data: ',len(test_images))
print('Imported training data: ', len(train_images))
print(class_names)


Imported test data:  400
Imported training data:  1600
['bedroom', 'coast', 'forest', 'kitchen', 'livingroom', 'mountain', 'office', 'store', 'street', 'tallbuilding']


###  2 Build Vocabulary and Clusters

Extract SIFT features from the training data using ***features.extract_dsift(..)*** and then utilize the descriptors to generate visual words with ***KMeans(...).fit(...).cluster_centers_*** from ***sklearn.neighbors***. Set the sample size per image at about **100** and a step size around **5**. For cluster size, try a value around **50**. You can adapt the paramater to achieve better results. If clustering takes too long, have a look at the parameters **n_init** and **max_iter** of ***KMeans(...)***.

_***HINT:***_
***KMeans(..)*** expects the features to be a stacked 2D matrix instead of a list (np.vstack(...)).


In [52]:
#student code start

descriptors = features.extract_dsift(train_images, stepsize=5, num_samples=100)
descriptors = np.array(descriptors)
print(f"descriptors dimension: {descriptors.shape}")

descriptors = np.vstack(descriptors)  # Stack descriptors from all images
print(f"Stacked descriptors dimension: {descriptors.shape}")


# Fit KMeans to the descriptors
kmeans = KMeans(n_clusters=50)
kmeans.fit(descriptors)

# Cluster centers represent the visual words
visualWords = kmeans.cluster_centers_

DSIFT Extraction: 7.44045510000069  seconds
descriptors dimension: (1600, 100, 128)
Stacked descriptors dimension: (160000, 128)


###  3 Classification

Generat a histogram per image using the centroids. First extract the dense SIFT features of the training set again. This time use a smaller stepsize (eg. 1 or 2) and take all samples.

After, implement ***count_visual_words(..)*** in ***features.py*** to obtain a histogram per image. Build a kNN classifier with ***KNeighborsClassifier(..)*** and fit the the model with the histogram data of the training images and training labels. Set the neighbourhood size to **3**.


In [61]:
#student code start
train_descriptors = features.extract_dsift(train_images, stepsize=1, num_samples=None)
histograms = features.count_visual_words(train_descriptors, visualWords)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(histograms, train_labels)
#student code end

DSIFT Extraction: 19.80681530000038  seconds
Counting visual words: 25.980671500000426  seconds


KNeighborsClassifier(n_neighbors=3)

### 4 Evaluation

Examine the accuracy of your model by extracting the dense SIFT points of the test set with the same stepsize as before, generate the histogams and predict the labels of the test set with the previously fitted classifier. Plot your results with ***utils.plot_confusion_matrix(..)***.

_***HINT:***_
Use the methods ***predict(..)*** and ***score(..)*** on the KNN classifier.


***Submission:*** Save the confusion matrix with the KNN score as **task3_accuracy_test_data.png** using ***utils.plot_confusion_matrix(..)***.



In [62]:
#student code start
test_descriptors = features.extract_dsift(test_images, stepsize=1, num_samples=None)
test_histograms = features.count_visual_words(test_descriptors, visualWords)

predictions = knn.predict(test_histograms)

accuracy = knn.score(test_histograms, test_labels)
print(f"Accuracy on test set: {accuracy * 100:.2f}%")

#student code end

DSIFT Extraction: 4.149897100000089  seconds
Counting visual words: 6.442469800000254  seconds
Accuracy on test set: 21.50%


***
#### Q1: Analyze the confusion matrix. Are there classes which can be more easily identified than others. What could be the reason? What classes perform worst, and explain your answer.


#ANSWER HERE


Take some own test images and investigate if they can be correctly classified with you trained model. For instance, take a photo of your own kitchen or living room and classify it. You should at least test one photo from three of the ten categories. Make sure the images are resized and cropped to 100x100.

***Submission:*** Save the confusion matrix with the model score as **task3_accuracy_custom_data.png** using ***utils.plot_confusion_matrix(..)***.

In [ ]:
#student code start

#student code end

### 5 Beyond Bag Of Words: Vector of Locally Aggregated Descriptors (VLAD)

Finally, we want to investigate an extension of the traditional bag of words model, called **Vector of Locally Aggregated Descriptors (VLAD)**, which not only considers the cluster assignment, but also uses the distance to the nearest centroid (refered to as **residual**).

Implement the VLAD algorithm in **features.calculate_vlad_descriptors** according to the task description and run the whole classification again by replacing the bag of words descriptors with VLAD. Compare the performance and discuss the advantages and disadvantages. Can you think of any further extensions?

***Submission:*** Save the confusion matrix with the model score as **task3_accuracy_vlad_descriptor.png** using ***utils.plot_confusion_matrix(..)***.

In [ ]:
#student code start

#student code end